# 10个范例带你入门LangChain 

5，数据库问答(Querying Tabular Data): 从数据库/类数据库内容中抽取数据信息。

In [1]:
# 使用你自己的OpenAI API key
openai_api_key='sk-PFwMvMFcU7VXaL5cYM9vT3B'
zhipu_api_key='08132b620644081b192567639fc0a9b1.OAz1XI2d21zLZDk9'

In [2]:
import time
import jwt
 
def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)
 
    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }
 
    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )

In [3]:
# from langchain_community.chat_models import ChatOpenAI
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
llm =ChatOpenAI(
        model_name="glm-4",
        openai_api_base="https://open.bigmodel.cn/api/paas/v4",
        openai_api_key=generate_token(zhipu_api_key,10000),
        streaming=False,
        temperature=0.5
    )

## 五，数据库问答(Querying Tabular Data)

In [4]:
# 使用自然语言查询一个 SQLite 数据库，我们将使用旧金山树木数据集
# Don't run following code if you don't run sqlite and follow db
from langchain import OpenAI, SQLDatabase
# %pip install  --user langchain_experimental
from langchain_experimental.sql import SQLDatabaseChain
# llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

In [5]:
sqlite_db_path = 'San_Francisco_Trees.db'
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

In [6]:
db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True)

提示词： input_variables=['input', 'table_info', 'top_k'] template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use date(\'now\') function to get the current date, if the question involves "today".\n\nUse the following

In [7]:
db_chain.invoke("How many Species of trees are there in San Francisco?")



> Entering new SQLDatabaseChain chain...
_run_manager <langchain_core.callbacks.manager.CallbackManagerForChainRun object at 0x7f7a09968580>
inputs： {'query': 'How many Species of trees are there in San Francisco?'}
inputs[self.input_key]： How many Species of trees are there in San Francisco?
input_text：
 How many Species of trees are there in San Francisco?
SQLQuery: :input_text
How many Species of trees are there in San Francisco?
SQLQuery:table_names_to_use： None
table_info： 
CREATE TABLE "SFTrees" (
	"TreeID" INTEGER, 
	"qSpecies" TEXT, 
	"qAddress" TEXT, 
	"PlantType" TEXT
)

/*
3 rows from SFTrees table:
TreeID	qSpecies	qAddress	PlantType
168225	Arbutus 'Marina' :: Hybrid Strawberry Tree	2547 Vallejo St	Tree
168228	Arbutus 'Marina' :: Hybrid Strawberry Tree	2547 Vallejo St	Tree
189160	Afrocarpus gracilior :: Fern Pine	100 Kissling St	Tree
*/
self.memory None
llm_inputs {'input': 'How many Species of trees are there in San Francisco?\nSQLQuery:', 'top_k': '5', 'dialect': 'sqlite

OperationalError: (sqlite3.OperationalError) unrecognized token: ":"
[SQL: SELECT COUNT(DISTINCT "qSpecies") FROM "SFTrees"
SQLResult:]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

1. Find which table to use
2. Find which column to use
3. Construct the correct sql query
4. Execute that query
5. Get the result
6. Return a natural language reponse back

confirm LLM result via pandas

In [ ]:
# import sqlite3
# %pip install pandas --user
import pandas as pd

# Connect to the SQLite database
connection = sqlite3.connect(sqlite_db_path)

# Define your SQL query
query = "SELECT count(distinct qSpecies) FROM SFTrees"

# Read the SQL query into a Pandas DataFrame
df = pd.read_sql_query(query, connection)

# Close the connection
connection.close()

/Users/davy/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


NameError: name 'sqlite3' is not defined

In [ ]:
# Display the result in the first column first cell
print(df.iloc[0,0])

578


In [8]:
llm_inputs={'input': 'How many Species of trees are there in San Francisco?\nSQLQuery:', 'top_k': '5', 'dialect': 'sqlite', 'table_info': '\nCREATE TABLE "SFTrees" (\n\t"TreeID" INTEGER, \n\t"qSpecies" TEXT, \n\t"qAddress" TEXT, \n\t"PlantType" TEXT\n)\n\n/*\n3 rows from SFTrees table:\nTreeID\tqSpecies\tqAddress\tPlantType\n168225\tArbutus \'Marina\' :: Hybrid Strawberry Tree\t2547 Vallejo St\tTree\n168228\tArbutus \'Marina\' :: Hybrid Strawberry Tree\t2547 Vallejo St\tTree\n189160\tAfrocarpus gracilior :: Fern Pine\t100 Kissling St\tTree\n*/', 'stop': ['\nSQLResult:']}

In [ ]:
sql_cmd = SQLDatabaseChain.llm_chain.predict(#此处通过提示词  生成sql查询语句。今日问题查至此处？？？？
        callbacks=_run_manager.get_child(),
        **llm_inputs,
    ).strip()